In [18]:
from textblob import TextBlob
import pandas as pd
import numpy as np
import httpcore,os,json,time

In [ ]:
class TraducaoMqV():
    def __init__(self,df):
        self.df = df
        pergunta_um = []
        pergunta_dois = []
    
    def fun_translater(pergunta):         
        loop = True
        while(loop):
            try:
                translator = TextBlob(pergunta)
                pergunta_br = translator.translate(to = 'pt')
                time.sleep(5)
                loop = False # Stop the loop
                return pergunta_br

            except Exception:
                time.sleep(10)
            
    def init_traducao(self):
        for i in range(463,self.df.shape[0]):
            try:
                pergunta_um_traduzida = self.fun_translater(self.df.loc[i][1])
                pergunta_dois_traduzida = self.fun_translater(self.df.loc[i][2])
                self.pergunta_um.append(pergunta_um_traduzida.string)
                self.pergunta_dois.append(pergunta_dois_traduzida.string)
                print("debug 1")

                with open('./backup/pergunta_um.json','w+') as outfile_um:
                    json.dump(self.pergunta_um,outfile_um,ensure_ascii=False)
                    outfile.close()

                with open('./backup/pergunta_dois.json','w+') as outfile_dois:
                    json.dump(self.pergunta_dois,outfile_dois,ensure_ascii=False)
                    outfile.close()

                f = open('output.txt','a')
                f.write("%d de %d\n"%(i,self.df.shape[0]))
                f.close()

                df_ = pd.DataFrame({'pergunta_um' : self.pergunta_um, 'pergunta_dois' : self.pergunta_dois})
                df_.to_csv('./backup/backupMQP.csv',index=False)
                
                print("%d de %d"%(i,self.df.shape[0]))

            except (KeyboardInterrupt or httpcore._exceptions.ConnectError) as x:

                f = open('output.txt','a')
                f.write("Traducao parada na posicao %d\n"%(i))
                f.write(str(x))
                f.close()

        df_save = pd.DataFrame({'pergunta_um' : self.pergunta_um, 'pergunta_dois' : self.pergunta_dois, 'label' : self.df['label']})
        df_save.to_csv('./mqp.csv',index=False)
        f = open('output.txt','a')
        f.write("Traducao concluida\n")
        f.close()
        
if('__main__' == __name__):

    df = pd.read_csv('./mqp.csv',names = ['dr_id', 'question_1', 'question_2', 'label'])

    path = './backup'
    if(not os.path.exists(path)):
        os.mkdir(path)
    
    print('oido')
    traducao_mqv = TraducaoMqV(df)
    traducao_mqv.init_traducao()

In [17]:
strArr = np.empty(10, dtype='s256')
strArr

TypeError: data type 's256' not understood

In [15]:
teste[0] = 'fdsjfnkdsjfn dsfknsdjfsnkd djkfnskjdfnskd'

IndexError: too many indices for array: array is 0-dimensional, but 1 were indexed

In [10]:
teste

chararray([b'f', b'3', b'\x97', ..., b'\x7f', '', ''], dtype='|S1')

In [1]:
'__main__' == __name__

True

In [5]:
df = pd.read_csv('./mqp.csv',names = ['dr_id', 'question_1', 'question_2', 'label'])

In [5]:
df.loc[0][2]

'I have a party tonight and I took my last dose of Azithromycin this morning. Can I have a few drinks?'

In [13]:
teste = [df[df.index == 463]['question_1'].values[0], df[df.index == 463]['question_2'].values[0]]


In [16]:
teste

['Weeping blisters ontop of foot,red toes...looks terrible.what to do??',
 "I think I have a weeping blister on toe, how do I know if it's a simple shoe bite or an infection?"]

In [33]:
def fun_translater(pergunta):         
    loop = True
    lim = 0
    while(loop):
        try:
            translator = TextBlob(pergunta)
            pergunta_br = translator.translate(to = 'pt')
            time.sleep(5)
            loop = False # Stop the loop
            return pergunta_br

        except Exception:
            time.sleep(10)

In [ ]:
while(True):
    try:
        translator = TextBlob(teste[0])
        pergunta_br = translator.translate(to = 'pt')
    except Exception or KeyboardInterrupt as x:
        print(x)
        

<class 'KeyboardInterrupt'>
